# Tutorial 4 - Memory

## Key Philosophy
- It would be important to learn from past experience and improve the agentic framework - memory is key to that
- You can add to the memory bank of your Agents pre-inference (by collecting from a pool of data prior to running the Agent), or during inference (add on in between running subtasks)

## Use Memory in Agents
- Agent class takes `memory_bank` as a parameter during initialisation of an `Agent`
- memory_bank: class Dict[Memory]. Stores multiple types of memory for use by the agent. Customise the Memory config within the Memory class.
    - Default: `memory_bank = {'Function': Memory(top_k = 5, mapper = lambda x: x.fn_description, approach = 'retrieve_by_ranker')}`
    - Key: `Function` (Already Implemented Natively) - Does RAG over Task -> Function mapping
    - Can add in more keys that would fit your use case. Retrieves similar items to task/overall plan (if able) for additional context in `get_next_subtasks()` and `use_llm()` function
    - Side Note: RAG can also be done (and may be preferred) as a separate function of the Agent to retrieve more information when needed (so that we do not overload the Agent with information)

## Memory Class
- Retrieves top k memory items based on task 
- Inputs:
    - `memory`: List. Default: Empty List. The list containing the memory items
    - `top_k`: Int. Default: 3. The number of memory list items to retrieve
    - `mapper`: Function. Maps the memory item to another form for comparison by ranker or LLM. Default: `lambda x: x`
        - Example mapping: `lambda x: x.fn_description` (If x is a Class and the string you want to compare for similarity is the fn_description attribute of that class)
    - `approach`: str. Either `retrieve_by_ranker` or `retrieve_by_llm` to retrieve memory items.
        - Ranker is faster and cheaper as it compares via embeddings, but are inferior to LLM-based methods for contextual information
    - `ranker`: `Ranker`. The Ranker which defines a similarity score between a query and a key. Default: OpenAI `text-embedding-3-small` model. 
        - Can be replaced with a function which returns similarity score from 0 to 1 when given a query and key
        
## Example Use Case
- Helps to reduce number of functions present in LLM context for more accurate generation
```python
output = my_agent.run('Calculate 2**10 * (5 + 1) / 10')
```

`Original Function List: add_numbers, subtract_numbers, add_three_numbers, multiply_numbers, divide_numbers, power_of, GCD_of_two_numbers, modulo_of_numbers, absolute_difference, generate_poem_with_numbers, List_related_words, generate_quote`

`Filtered Function Names: add_three_numbers, multiply_numbers, divide_numbers, power_of, modulo_of_numbers`

In [1]:
# !pip install taskgen-ai

In [2]:
# Set up API key and do the necessary imports
import os
from taskgen import *
import math

os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

# Use Case 1: Filtering Functions by Task
- TaskGen chooses `top k` (default k: 5) functions according to similarity to subtask
- In addition to `top k` functions, we will also give agent all the compulsory functions
    - `is_compulsory` variable of Function set to `True` means that we will always have it as one of the functions for planning and bypass Function RAG

In [3]:
# Define multiple functions
fn_list = [
    Function("Adds all numbers in <num_list: List[int]>", output_format = {"Result": "Sum of all numbers in num_list"}, 
        is_compulsory = True, external_fn = lambda num_list: sum(x for x in num_list), fn_name = 'sum_numbers'),
    Function("Subtracts <num1: float> from <num2: float>", output_format = {"Result": "num1 - num2"}, fn_name = 'subtract_numbers', external_fn = lambda num1, num2: num1 + num2),
    Function("Multiply <num1: float> by <num2: float>", output_format = {"Result": "num1 * num2"}, fn_name = 'multiply_numbers', external_fn = lambda num1, num2: num1 * num2),
    Function("Divide <num1: float> by <num2: float>", output_format = {"Result": "num1 / num2"}, fn_name = 'divide_numbers', external_fn = lambda num1, num2: num1 / num2),
    Function("Returns <num1: float>**<num2: float>", output_format = {"Result": "num1**num2"}, fn_name = 'power_operation', external_fn = lambda num1, num2: math.pow(num1,num2)),
    Function("Returns Greatest Common Divisor of <num1: int> and <num2: int>", output_format = {"Result": "GCD(num1, num2)"}, fn_name = 'greatest_common_divisor', external_fn = lambda num1, num2: math.gcd(num1, num2)),
    Function("Returns modulo of <num1: int> over <num2: int>", output_format = {"Result": "num1 % num2"}, fn_name = 'modulo_of_numbers', external_fn = lambda num1, num2: num1 % num2),
    Function("Returns absolute difference between <num1: float> and <num2: float>", output_format = {"Result": "abs(num1 - num2)"}, fn_name = 'absolute_difference', external_fn = lambda num1, num2: math.abs(num1-num2)),
    Function("Generates a poem containing <num1: float> and <num2: float>", output_format = {"Poem": "Poem"}, fn_name = 'generate_poem_with_numbers'),
    Function("Lists out <num: int> words related to <word: str>", output_format = {"List of words": "List of words, type: list"}, fn_name = 'list_related_words'),
    Function("Generates a quote about <topic: str>", output_format = {"Quote": "Quote"}, fn_name = 'generate_quote')
]

In [4]:
my_agent = Agent('Generalist Agent', 
'''Does everything''',
                default_to_llm = False).assign_functions(fn_list)

In [5]:
# see the auto-generated names of your functions :)
my_agent.list_functions()

['Name: end_task\nDescription: Passes the final output to the user\nInput: []\nOutput: {}\n',
 "Name: sum_numbers\nDescription: Adds all numbers in <num_list: List[int]>\nInput: ['num_list']\nOutput: {'Result': 'Sum of all numbers in num_list'}\n",
 "Name: subtract_numbers\nDescription: Subtracts <num1: float> from <num2: float>\nInput: ['num1', 'num2']\nOutput: {'Result': 'num1 - num2'}\n",
 "Name: multiply_numbers\nDescription: Multiply <num1: float> by <num2: float>\nInput: ['num1', 'num2']\nOutput: {'Result': 'num1 * num2'}\n",
 "Name: divide_numbers\nDescription: Divide <num1: float> by <num2: float>\nInput: ['num1', 'num2']\nOutput: {'Result': 'num1 / num2'}\n",
 "Name: power_operation\nDescription: Returns <num1: float>**<num2: float>\nInput: ['num1', 'num2']\nOutput: {'Result': 'num1**num2'}\n",
 "Name: greatest_common_divisor\nDescription: Returns Greatest Common Divisor of <num1: int> and <num2: int>\nInput: ['num1', 'num2']\nOutput: {'Result': 'GCD(num1, num2)'}\n",
 "Name: 

In [6]:
# visualise how the Functions are chosen based on task - here you see subtract_numbers appearing at the front
# this does not include the compulsory functions
[f.fn_name for f in my_agent.memory_bank['Function'].retrieve_by_ranker('subtract numbers')]

['subtract_numbers',
 'divide_numbers',
 'multiply_numbers',
 'modulo_of_numbers',
 'generate_poem_with_numbers']

In [7]:
my_agent.reset()
my_agent.run('Evaluate 2+3')

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo_of_numbers
Observation: No subtasks completed yet for Assigned Task
Thoughts: Use the sum_numbers function to add 2 and 3 together
Subtask identified: Adds all numbers in [2, 3]
Calling function sum_numbers with parameters {'num_list': [2, 3]}
> {'Result': 5}

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo_of_numbers
Observation: The task is to evaluate 2+3, which has been partially completed by adding all numbers in [2, 3] resulting in 5
Thoughts: The final step is to provide the sum of 2 and 3 to the user
Subtask identified: End Task
Task completed successfully!



[{'Result': 5}]

In [8]:
my_agent.reset()
output = my_agent.run('Evaluate 2**10 * (3+5) / 10')

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo_of_numbers
Observation: No subtasks completed yet
Thoughts: Need to start evaluating the expression provided by the user
Subtask identified: Returns 2**10
Calling function power_operation with parameters {'num1': 2.0, 'num2': 10.0}
> {'Result': 1024.0}

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo_of_numbers
Observation: The subtask "Returns 2**10" has been completed successfully, resulting in 1024.0
Thoughts: The remaining part of the assigned task is to multiply the result by (3+5) and then divide the final result by 10.
Subtask identified: Multiply 1024.0 by 8.0
Calling function multiply_numbers with parameters {'num1': 1024.0, 'num2': 8.0}
> {'Result': 8192.0}

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo

In [9]:
my_agent.reply_user()

819.2


819.2

In [10]:
my_agent.status()

Agent Name: Generalist Agent
Agent Description: Does everything
Available Functions: ['end_task', 'sum_numbers', 'subtract_numbers', 'multiply_numbers', 'divide_numbers', 'power_operation', 'greatest_common_divisor', 'modulo_of_numbers', 'absolute_difference', 'generate_poem_with_numbers', 'list_related_words', 'generate_quote']
Shared Variables: ['agent']
Task: Evaluate 2**10 * (3+5) / 10
Subtasks Completed:
Subtask: Returns 2**10
{'Result': 1024.0}

Subtask: Multiply 1024.0 by 8.0
{'Result': 8192.0}

Subtask: Divide 8192.0 by 10
{'Result': 819.2}

Subtask: Multiply 8192.0 by 8.0
{'Result': 65536.0}

Subtask: Add all numbers in [3, 5], then multiply the result by 2^10.
{'Result': 8}

Subtask: Evaluate 2**10 * (3+5) / 10
819.2

Is Task Completed: False


# Use Case 2: Adding more context based on task
- You can add additional context as the memory_bank so that `top k` will be added to prompt based on task

### Using `memory_bank` for more context
- Here, we have a mapping of nonsense words to numbers
- Based on the subtask, we will augment the system prompt with relevant mappings to aid planning

In [11]:
# first append more context to the agent
my_agent.memory_bank['Word to Numbers'] = Memory([{'Azo': 1}, {'Boneti': 2}, {'Andkh': 3}, {'Bdakf': 4}, {'dafdsk': 5}, 
            {'ldsfn': 6}, {'sdkfn': 7}, {'eri': 8}, {'knewro': 9}, {'mdsnfk': 10}], # some nonsense words
            top_k = 5,  # choose top 5
            mapper = lambda x: list(x.keys())) # we compare with the task using only the first word, e.g. Azo, Boneti, Andkh

In [12]:
my_agent.reset()
output = my_agent.run('Tell me what is Boneti + mdsnfk + Azo')

Filtered Function Names: end_task, sum_numbers, multiply_numbers, power_operation, modulo_of_numbers, generate_poem_with_numbers, generate_quote
Observation: No subtasks completed yet.
Thoughts: Need to extract the numbers corresponding to the words in the Assigned Task and perform the required operation.
Subtask identified: Extract the numbers corresponding to the words "Boneti", "mdsnfk", and "Azo" from the Knowledge Reference.
Calling function sum_numbers with parameters {'num_list': [2, 10, 1]}
> {'Result': 13}

Filtered Function Names: end_task, sum_numbers, multiply_numbers, power_operation, modulo_of_numbers, generate_poem_with_numbers, generate_quote
Observation: Extract the numbers corresponding to the words "Boneti", "mdsnfk", and "Azo" from the Knowledge Reference.
Thoughts: The numbers corresponding to the words "Boneti", "mdsnfk", and "Azo" have been extracted successfully. The next step is to add these numbers together to get the final result.
Subtask identified: Adds all

In [13]:
my_agent.reply_user()

The sum of Boneti, mdsnfk, and Azo is 13.


'The sum of Boneti, mdsnfk, and Azo is 13.'

## Alternative approach of providing additional information via a function
- Sometimes it might be better off to do the additional information providing in a separate function
- This helps us to do more specific augmentation, like doing RAG over documents, or doing rule-based augmentation
- This also helps to reduce the context length of the planner by offloading the augmentation to another function

In [14]:
# delete number meanings additional context memory to showcase the information providing function
if 'Number Meanings' in my_agent.memory_bank:
    del my_agent.memory_bank['Number Meanings']

In [15]:
# add in additional information function
# can also do RAG here if needed
def convert_word_to_number(list_of_words: list):
    '''Gets additional information about all unknown words in user_query'''
    word_to_numbers = {'Azo': 1, 'Boneti': 2, 'Andkh': 3, 'Bdakf': 4, 'dafdsk': 5, 
               'ldsfn': 6, 'sdkfn': 7, 'eri': 8, 'knewro': 9, 'mdsnfk': 10}
    
    output_string = ''
    
    list_of_words = str(list_of_words).lower()
    for key, value in word_to_numbers.items():
        if key.lower() in list_of_words.lower():
            output_string += f'{key} equals {value}, '
    return output_string

info_fn = Function('Gets additional information about all unknown words in <list_of_words: List[str]>',
                  output_format = {'Output': 'str'},
                  is_compulsory = True, #makes this function always available for agent
                  external_fn = convert_word_to_number)

info_fn(['Boneti', 'mdsnfk', 'Azo'])

{'Output': 'Azo equals 1, Boneti equals 2, mdsnfk equals 10, '}

In [16]:
# Assign newest function 
my_agent.assign_functions([info_fn])

In [17]:
my_agent.reset()
output = my_agent.run('Tell me what is Boneti + mdsnfk + Azo')

Filtered Function Names: end_task, sum_numbers, multiply_numbers, power_operation, modulo_of_numbers, generate_poem_with_numbers, generate_quote, convert_word_to_number
Observation: No subtasks completed yet
Thoughts: Need to identify the unknown words in the expression "Boneti + mdsnfk + Azo" and convert them to numbers
Subtask identified: Get additional information about all unknown words in the expression "Boneti + mdsnfk + Azo"
Calling function convert_word_to_number with parameters {'list_of_words': ['Boneti', 'mdsnfk', 'Azo']}
> {'Output': 'Azo equals 1, Boneti equals 2, mdsnfk equals 10, '}

Filtered Function Names: end_task, sum_numbers, multiply_numbers, power_operation, modulo_of_numbers, generate_poem_with_numbers, generate_quote, convert_word_to_number
Observation: Get additional information about all unknown words in the expression "Boneti + mdsnfk + Azo"
Thoughts: The unknown words in the expression have been identified and their corresponding numerical values have been d

In [18]:
my_agent.reply_user()

Azo equals 1, Boneti equals 2, mdsnfk equals 10. Therefore, Boneti + mdsnfk + Azo equals 2 + 10 + 1, which equals 13.


'Azo equals 1, Boneti equals 2, mdsnfk equals 10. Therefore, Boneti + mdsnfk + Azo equals 2 + 10 + 1, which equals 13.'

# Use Case 3: Adding more context of when to call various functions
- We can augment memory bank with information of what functions to call for certain queries, so that agent knows what to do for some edge cases
- This helps to augment with the system prompt of `get_next_subtask` and `use_llm` with more examples related to the user query

In [19]:
my_agent.memory_bank['Task to Function'] = Memory([
    {'Task': 'Evaluate Azo + eri', 'Function1': {'name': 'convert_word_to_number', 'list_of_words': ['Azo', 'eri']}, 
     'Function2': {'name': 'sum_numbers_in_list', 'num_list': [1, 8]}},
    {'Task': 'Evaluate 5 + 2', 'Function': 'sum_numbers_in_list', 'num_list': [5, 2]},
    {'Task': 'Find out about Boneti and Andkh', 'Function': 'convert_word_to_number', 'list_of_words': ['Boneti', 'Andkh']},
    {'Task': 'What is Andkh?', 'Function': 'convert_word_to_number', 'list_of_words': ['Andkh']},
    {'Task': 'Booyah', 'Function': 'generate_quote', 'topic': 'TaskGen'}
      ], 
    top_k = 3,  # choose top 3
    mapper = lambda x: x['Task']) # we compare with the task using only with the user query

In [20]:
my_agent.reset()
output = my_agent.run('Tell me what is Boneti + mdsnfk + Azo')

Filtered Function Names: end_task, sum_numbers, multiply_numbers, power_operation, modulo_of_numbers, generate_poem_with_numbers, generate_quote, convert_word_to_number
Observation: No subtasks completed yet for the Assigned Task
Thoughts: Need to find the word to number conversion for Boneti, mdsnfk, and Azo
Subtask identified: Convert the words Boneti, mdsnfk, and Azo to numbers
Calling function convert_word_to_number with parameters {'list_of_words': ['Boneti', 'mdsnfk', 'Azo']}
> {'Output': 'Azo equals 1, Boneti equals 2, mdsnfk equals 10, '}

Filtered Function Names: end_task, sum_numbers, multiply_numbers, power_operation, modulo_of_numbers, generate_poem_with_numbers, generate_quote, convert_word_to_number
Observation: The words Boneti, mdsnfk, and Azo have been converted to numbers: Azo equals 1, Boneti equals 2, mdsnfk equals 10, 
Thoughts: The task is to find the sum of Boneti, mdsnfk, and Azo. This can be achieved by using the sum_numbers function.
Subtask identified: Add th

In [21]:
my_agent.reply_user()

Boneti + mdsnfk + Azo equals 2 + 10 + 1 which equals 13


'Boneti + mdsnfk + Azo equals 2 + 10 + 1 which equals 13'

In [22]:
my_agent.reset()
# This actually is not anything special - but because of memory bank, it is mapped to TaskGen
output = my_agent.run('Booyah')

Filtered Function Names: end_task, sum_numbers, multiply_numbers, divide_numbers, power_operation, modulo_of_numbers, generate_quote, convert_word_to_number
Observation: No subtasks completed yet for Assigned Task
Thoughts: Need to start working on the Assigned Task
Subtask identified: Generate a quote about the topic "TaskGen"
Calling function generate_quote with parameters {'topic': 'TaskGen'}
> {'Quote': 'TaskGen is a powerful tool that simplifies the process of generating tasks and managing workflows.'}

Filtered Function Names: end_task, sum_numbers, multiply_numbers, divide_numbers, power_operation, modulo_of_numbers, generate_quote, convert_word_to_number
Observation: The quote "TaskGen is a powerful tool that simplifies the process of generating tasks and managing workflows." has been generated for the topic "TaskGen".
Thoughts: The next subtask is "End Task".
Subtask identified: End Task
Task completed successfully!



In [23]:
my_agent.status()

Agent Name: Generalist Agent
Agent Description: Does everything
Available Functions: ['end_task', 'sum_numbers', 'subtract_numbers', 'multiply_numbers', 'divide_numbers', 'power_operation', 'greatest_common_divisor', 'modulo_of_numbers', 'absolute_difference', 'generate_poem_with_numbers', 'list_related_words', 'generate_quote', 'convert_word_to_number']
Shared Variables: ['agent']
Task: Booyah
Subtasks Completed:
Subtask: Generate a quote about the topic "TaskGen"
{'Quote': 'TaskGen is a powerful tool that simplifies the process of generating tasks and managing workflows.'}

Is Task Completed: True


# Comparison between embedding-based methods using Ranker and LLM-based similarity
- Pros of embedding-based similarity comparison: Fast and cheap
- Cons of embedding-based similarity comparison: Not as accurate
- If using default `Ranker` function (OpenAI embedding model), automatically stores new embeddings generated in `database` and uses back known embeddings from `database` when possible, potentially helping to save time and costs
- Select the right method for your use case

- (Advanced Exercise) Instead of using cosine similarity using OpenAI Embeddings, create your own `ranking_fn` within `Ranker` that does similarity search the way you want it to

In [24]:
database = {}
memory = Memory(['hello', 'no worries', 'goodbye', 'hurray'], top_k = 1, ranker = Ranker(database = database))
print('Using embeddings', memory.retrieve_by_ranker('Another word for hi'))
print('Using LLM', memory.retrieve_by_llm('Another word for hi'))

Using embeddings ['hello']
Using LLM ['hello']


In [25]:
# Visualise the keys in the database
database.keys()

dict_keys(['hello', 'Another word for hi', 'no worries', 'goodbye', 'hurray'])

In [26]:
print('Using embeddings', memory.retrieve_by_ranker('What to say when leaving'))
print('Using LLM', memory.retrieve_by_llm('What to say when leaving'))

Using embeddings ['goodbye']
Using LLM ['goodbye']


In [27]:
# Visualise the keys in the database
database.keys()

dict_keys(['hello', 'Another word for hi', 'no worries', 'goodbye', 'hurray', 'What to say when leaving'])